## **#07. Panel Data Analysis**
- Instructor: [Jaeung Sim](https://jaeungs.github.io/) (University of Connecticut)
- Course: OPIM 5671 Data Mining and Time Series Forecasting
- Last updated: November 11, 2025

**Objectives**
* Apply causal inference methods to real-world data.

**References**
* [Frontiers: Virus Shook the Streaming Star: Estimating the COVID-19 Impact on Music Consumption](https://pubsonline.informs.org/doi/abs/10.1287/mksc.2021.1321)

##### **Part 0. Basic Setup**

In [ ]:
# Set your Google Drive directory
import os
os.getcwd()

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/Colab Notebooks/OPIM 5671 (Fall 2025)') # You may need to change this directory

In [ ]:
# Import basic data-processing and visualization libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##### **Part 1. Exploratory Data Analysis**

We are going to explore the real-world dataset used in the following paper:
> Sim, J., Cho, D., Hwang, Y., & Telang, R. (2022). Frontiers: virus shook the streaming star: estimating the COVID-19 impact on music consumption. *Marketing Science*, 41(1), 19-32.

**Basic Data Structure**
* The paper used Spotify's weekly streaming data for the top 200 weekly music charts during 104 consecutive weeks between June 2018 and May 2020 across 60 countries.
* They aggregated weekly streaming counts of the top 200 songs for each country and built the country-week level for 104 weeks of 60 countries, for a total of 6,240 observations.

**Combined Datasets**
* They combined music consumption data with COVID-19 case statistics from the European Centre for Disease Prevention and Control (ECDC).
* They utilized data on enforced social distancing measures by the governments from the Oxford COVID-19 Government Response Tracker.
* Also, they used country-day-level data on individuals' time allocation changes (for activities) since the COVID-19 outbreak from Google's COVID-19 Community Mobility Reports.

**1.1. Exploring Raw Data**

In [ ]:
# Load a dataset from an MS Excel sheet
df = pd.read_csv('dataset_07_covid_music.csv')
df.head(10)

In [ ]:
# Divide 41 columns into two parts (1 of 2)
df[df.columns[0:20]].head(10)

In [ ]:
# Divide 41 columns into two parts (2 of 2)
df[df.columns[21:41]].head(10)

**1.2. Exploring Summary Statistics**

In [ ]:
# Summary Statistics
df.describe()

In [ ]:
# Divide 41 columns into two parts (1 of 2)
df[df.columns[0:20]].describe()

In [ ]:
# Divide 41 columns into two parts (2 of 2)
df[df.columns[21:41]].describe()

**1.3. Visualization**

Please refer to **Figure 2** in the paper.

In [ ]:
# GroupBy at the (treated year x week of period) level
streams_by_treated = df.groupby(['treated','week_of_period'])['num_streams'].sum()
streams_by_treated = streams_by_treated.reset_index()
streams_by_treated.head(10)

In [ ]:
# Filter the DataFrame for treated and untreated observations
untreated = streams_by_treated[streams_by_treated['treated'] == 0]
treated = streams_by_treated[streams_by_treated['treated'] == 1]

# Specify x and y axis
plt.plot(untreated['week_of_period'], untreated['num_streams'], label='Untreated', color='red')
plt.plot(treated['week_of_period'], treated['num_streams'], label='Treated', color='blue')

# Add labels and title
plt.xlabel('Week of Period')
plt.ylabel('Streams')
plt.title('Weekly Streams by Group')
plt.legend()

# Show plot
plt.show()

In [ ]:
# Filter the DataFrame for treated and untreated observations
untreated = streams_by_treated[streams_by_treated['treated'] == 0]
treated = streams_by_treated[streams_by_treated['treated'] == 1]

# Create the figure and the first axes (the left y-axis)
fig, ax1 = plt.subplots()

# Plot the first line
ax1.plot(untreated['week_of_period'], untreated['num_streams'], color='blue', label='Untreated')
ax1.set_xlabel('Time')
ax1.set_ylabel('Streams (Untreated)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create the second axes (the right y-axis) based on the first
ax2 = ax1.twinx()

# Plot the second line
ax2.plot(treated['week_of_period'], treated['num_streams'], color='red', label='Treated')
ax2.set_ylabel('Streams (Treated)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Add title
plt.title('Weekly Streams by Group')

# Show plot
plt.show()


Compare the results with **Figure 3** in the paper.

##### **Part 2. Econometric Analysis**

**2.1. Basic Difference-in-Differences with `PanelOLS` in `linearmodels`**


In [ ]:
# Install `linearmodels` library
!pip install linearmodels

In [ ]:
# Import `linearmodels` library
from linearmodels import PanelOLS

In [ ]:
# Setting a new index by combining 'country_index' and 'treated'
df['new_id'] = df['country_index'] + '_' + df['treated'].astype(str)

# Creating an interaction term 'treated_after' by multiplying 'treated' by 'after'
df['treated_after'] = df['treated'] * df['after']

In [ ]:
# Setting multi-index for panel data
df['week_of_period_id'] = df['week_of_period']
df = df.set_index(['new_id', 'week_of_period_id'])

Basic functional form:

$y_{ijt} = \alpha_i + \beta_1 \cdot Treated_j + \beta_2 \cdot After_t + \beta_3 \cdot Treated_j \cdot After_t + \Sigma_t \delta_t + \varepsilon_{ijt}$

Because we set each combination of country-year ($ij$) as an entity, the equation becomes as:

$y_{ijt} = \beta_1 \cdot Treated_j + \beta_2 \cdot After_t + \beta_3 \cdot Treated_j \cdot After_t + \Sigma_i \Sigma_j \alpha_i \cdot \gamma_j + \Sigma_t \delta_t + \varepsilon_{ijt}$

In [ ]:
# Fitting the Panel regression model with two-way fixed effects
mod = PanelOLS.from_formula('num_streams ~ treated + after + treated_after + EntityEffects + TimeEffects', data=df) # Don't automatically drop variables based on multicollinearity
res = mod.fit(cov_type='clustered', cluster_entity=True) # Clustered residual errors

# Printing the model's summary
print(res)

In [ ]:
# Fitting the Panel regression model with two-way fixed effects
mod = PanelOLS.from_formula('num_streams ~ treated_after + EntityEffects + TimeEffects', data=df) # Dropping fully absorbed variables
res = mod.fit(cov_type='clustered', cluster_entity=True) # Clustered residual errors

# Printing the model's summary
print(res)

In [ ]:
# Fitting the Panel regression model with two-way fixed effects
mod = PanelOLS.from_formula('ln_num_streams ~ treated_after + EntityEffects + TimeEffects', data=df) # Using a log-transformed dependent variable
res = mod.fit(cov_type='clustered', cluster_entity=True) # Clustered residual errors

# Printing the model's summary
print(res)

**2.2. High-dimensional Fixed Effects with `pyhdfe` and `statsmodel.api`**


In [ ]:
# Necessary libraries
import pyhdfe
import statsmodels.api as sm

New functional form:

$y_{ijt} = \beta_1 \cdot Treated_j + \beta_2 \cdot After_t + \beta_3 \cdot Treated_j \cdot After_t + \Sigma_i \Sigma_j \alpha_i \cdot \gamma_j + \Sigma_j \Sigma_t \gamma_j \cdot \delta_t + \Sigma_i \Sigma_t \alpha_i \cdot \delta_t +  \varepsilon_{ijt}$

Here, you should introduce three types of two-dimensional fixed effects

In [ ]:
# Define the regressors (not fixed effects)
X = df[['treated_after']].values
y = df[['ln_num_streams']].values

In [ ]:
# Create `ij`, `jt` and `it` combinations
df['ij_FE'] = df['country_index'] + '_' + df['treated'].astype(str)
df['jt_FE'] = df['treated'].astype(str) + '_' + df['week_of_period'].astype(str)
df['it_FE'] = df['country_index'] + '_' + df['week_of_period'].astype(str)

In [ ]:
# Create the fixed effects matrix
fe_matrix = df[['ij_FE', 'jt_FE', 'it_FE']].values

In [ ]:
# Create HDFE object
hdfe = pyhdfe.create(fe_matrix)

In [ ]:
print(X.shape)
print(y.shape)
print(fe_matrix.shape)

In [ ]:
# Absorb fixed effects (demean the variables)
X_demeaned = hdfe.residualize(X)
y_demeaned = hdfe.residualize(y)

In [ ]:
# Fit OLS on demeaned data
X_demeaned_constant = sm.add_constant(X_demeaned)
model = sm.OLS(y_demeaned, X_demeaned_constant).fit()

In [ ]:
# Show summary
print(model.summary()) # Fully absorbed by fixed effects

It's time to incorporate COVID-19 statistics!

$y_{ijt} = \theta_1 \cdot Cases_{ijt} + \theta_2 \cdot Deaths_{ijt} + \Sigma_i \Sigma_j \alpha_i \cdot \gamma_j + \Sigma_j \Sigma_t \gamma_j \cdot \delta_t + \Sigma_i \Sigma_t \alpha_i \cdot \delta_t +  \varepsilon_{ijt}$


In [ ]:
X = df[['cases_per_million', 'deaths_per_million']].values

In [ ]:
# Absorb fixed effects (demean the variables)
X_demeaned = hdfe.residualize(X)
y_demeaned = hdfe.residualize(y)

In [ ]:
# Fit OLS on demeaned data
X_demeaned_constant = sm.add_constant(X_demeaned)
model = sm.OLS(y_demeaned, X_demeaned).fit()

In [ ]:
# Show summary
print(model.summary()) # Fully absorbed by fixed effects

End of Document